<a href="https://colab.research.google.com/github/Abbhiraami/algorithm_scratch/blob/main/dt_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Decision Tree from scratch

In [1]:
import pandas as pd
import numpy as np

In [2]:
### 1. Let's set up a sample dataset for building a Decision Tree
# Dataset
data = {
    'Weather': ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain', 'Rain', 'Overcast', 'Sunny', 'Sunny', 'Rain', 'Sunny', 'Overcast', 'Overcast', 'Rain'],
    'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild'],
    'PlayTennis': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']
}

df = pd.DataFrame(data)
df

,Weather,Temperature,PlayTennis
0,Sunny,Hot,No
1,Sunny,Hot,No
2,Overcast,Hot,Yes
3,Rain,Mild,Yes
4,Rain,Cool,Yes
5,Rain,Cool,No
6,Overcast,Cool,Yes
7,Sunny,Mild,No
8,Sunny,Cool,Yes
9,Rain,Mild,Yes


Entropy is a measure of uncertainty in the dataset. This is one of the way to
understand the mix of classes in the dataset.

Entropy Formula,

$Entropy(S)=-∑ p_i * log_2 p_i$

In [73]:
### 2. Calculate entropy
def entropy(target_variable):
  classes,counts=np.unique(target_variable,return_counts=True)
  # print(classes,counts)
  entropy=0
  for i in range(len(classes)):
    p_i=counts[i]/np.sum(counts)
    log2_pi=np.log2(p_i)
    ### Summation of negative values
    entropy -= p_i*log2_pi
  return entropy
print(f'Entropy of the synthetic weather dataset:{entropy(df["PlayTennis"]):1f}')
entropy(dummy.Y)

Entropy of the synthetic weather dataset:0.940286


1.584962500721156

Information Gain: Gives the best feature to split the dataset on. It measures the reduction in entropy after the dataset is split based on a feature

Information Gain = $Entropy(S) - ∑ \frac{|S_v|}{|S|} * Entropy(S_v)$


In short, Information gain = total entropy - weighted entropy

total entropy is the entropy of the target column and weighted entropy is the entropy of each feature set classes

The feature with the highest information gain will be selected as the feature to split the data

In [87]:
def information_gain(data,feature,target):
  ### entropy of target varaible ###
  total_entropy=entropy(data[target])
  ### Distinct values of features with their respective counts ###
  feat,counts=np.unique(data[feature],return_counts=True)
  # print(feat)

  ### Calculate the weighted entropy of each subset of a feature ###
  weighted_entropy=0
  ### Information gain
  for i in range(len(feat)):
    # print(i)
    subset=data[data[feature]==feat[i]]
    prob=counts[i]/np.sum(counts)
    weighted_entropy +=  prob* entropy(subset[target])
  # print(weighted_entropy)
  return total_entropy - weighted_entropy

### Calculating the information gain for all features
info_gain_df=pd.DataFrame()
for j in df[['Weather','Temperature']].columns:
  # print(j)
  info_gain=information_gain(df,j,'PlayTennis')
  gain_df=pd.DataFrame({'feature':[j],'information_gain':[info_gain]})
  info_gain_df=pd.concat([info_gain_df,gain_df]).reset_index(drop=True)

info_gain_df

print(f'The feature for split: {info_gain_df["feature"][np.argmax(info_gain_df.information_gain)]} with the information gain: {np.max(info_gain_df.information_gain)}')
info_gain_df

The feature for split: Weather with the information gain: 0.24674981977443933


,feature,information_gain
0,Weather,0.246750
1,Temperature,0.029223


4. Build a Decision Tree

Now, let's build a decision tree,

1. Calculate information gain
2. Select the best feature to split the data
3.



In [91]:
def decision_tree(data,original_data,features,target_variable,parent_node_class=None,max_features=None):
  # If all target values are same, return the class
  if len(np.unique(data[target_variable]))<=1:
    return np.unique(data[target_variable])[0]
  # If the dataset is empty, return the mode of the original data
  elif len(data)==0:
    return np.unique(original_data[target_variable])[np.argmax(np.unique(original_data[target_variable], return_counts=True)[1])]
  elif len(features)==0:
    return parent_node_class
  else:
    #store the mode of the target in the parent_node_class
    parent_node_class=np.unique(data[target_variable])[np.argmax(np.unique(original_data[target_variable],return_counts=True)[1])]

    ### Choosing the best feature to split the data
    item_values=[information_gain(data,feature,target_variable) for feature in features]
    best_feature_index=np.argmax(item_values)
    best_feature=features[best_feature_index]

    # Create the tree structure
    tree={best_feature:{}}

    # Remove the best feature
    features=[i for i in features if i !=best_feature]

    for value in np.unique(data[best_feature]):
      # print(value)
      sub_data=data[data[best_feature]==value]
      subtree=decision_tree(sub_data,original_data,features,target_variable,parent_node_class)
      tree[best_feature][value]=subtree
    return tree

In [92]:
tree=decision_tree(df,df,df.columns[:-1],df.columns[-1])
tree

{'Weather': {'Overcast': 'Yes',
  'Rain': {'Temperature': {'Cool': 'Yes', 'Mild': 'Yes'}},
  'Sunny': {'Temperature': {'Cool': 'Yes', 'Hot': 'No', 'Mild': 'Yes'}}}}

In [93]:
def predict(query,tree):
  for key in query.keys():
    if key in tree:
      result=tree[key][query[key]]
      if isinstance(result,dict):
        return predict(query,result)
      else:
        return result

### Prediction
query={'Weather':'Rain','Temperature':'Mild'}
predict(query,tree)

'Yes'

Improvising the decision tree to Random forest
Concept of Random Forest

1. Bootstrap Sampling: Each tree is trained on a random sample (with replacement) of the training data.
2. Feature Randomness: Each node in a tree is split based on a random subset of features, adding diversity to the model and reducing overfitting.
3. Aggregation: After building multiple trees, predictions are aggregated (majority voting for classification or averaging for regression).

Random Forest Pseudocode:
For each tree:

*   Take a random sample from the dataset (with replacement).

*   Build a decision tree using the sample.

*   Limit the number of features to consider at each split.

To make predictions:

*   Aggregate the predictions from all trees.


In [94]:
def bootstrapping(data):
  n_samples=data.shape[0]
  indices=np.random.choice(n_samples,size=n_samples,replace=True)
  return data.iloc[indices]

bootstrapping(df).shape



(14, 3)

In [102]:
class RandomForest:
    def __init__(self, n_trees, max_features=None):
        self.n_trees = n_trees
        self.max_features = max_features
        self.trees = []

    def fit(self, data, target):
        features = data.columns.drop(target)
        for _ in range(self.n_trees):
            # Create a bootstrapped dataset

            bootstrapped_data = bootstrapping(data)
            # print(bootstrapped_data)
            # Build a decision tree with a subset of features
            tree = decision_tree(bootstrapped_data, bootstrapped_data, features, target, max_features=self.max_features)
            self.trees.append(tree)
    print(tree)
    def predict(self, query):
        # Get predictions from each tree and return the majority vote
        predictions = []
        for tree in self.trees:
          prediction = predict(query, tree)
          predictions.append(prediction)

        # Majority voting for classification
        return max(set(predictions), key=predictions.count)

# Example of building and predicting with the Random Forest
random_forest = RandomForest(n_trees=10, max_features=2)
random_forest.fit(df, 'PlayTennis')

# Example prediction with a query
query = {'Weather': 'Overcast', 'Temperature': 'Cool'}
random_forest.predict(query)  # Output: 'Yes' or 'No'


{'Weather': {'Overcast': 'Yes', 'Rain': {'Temperature': {'Cool': 'Yes', 'Mild': 'Yes'}}, 'Sunny': {'Temperature': {'Cool': 'Yes', 'Hot': 'No', 'Mild': 'Yes'}}}}


'Yes'